In [1]:
import cv2
import numpy as np

In [2]:
cap = cv2.VideoCapture('Night Time Traffic Camera video.mp4') 
vehicle =0
matches =[]

def get_center(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)

    cx = x + x1
    cy = y + y1
    return cx,cy

object_detector = cv2.createBackgroundSubtractorMOG2(history= 100, varThreshold=200)

In [3]:
while True:
    ret, frame = cap.read()
     
        
    height, width, _ = frame.shape


    roi= frame[100: , 500: ]

    mask = object_detector.apply(roi)
    _, mask= cv2.threshold(mask, 253.5, 254.5, cv2.THRESH_BINARY)
    kernel = np.ones((5,5), np.uint8)

    img_dilation = cv2.dilate(mask, kernel, iterations=1)
    img_erosion = cv2.erode(img_dilation, kernel, iterations=1)
    img_erosion = cv2.medianBlur(img_erosion, 7)

    contours , _ = cv2.findContours(img_erosion, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    detections= []

    cv2.line (roi, (0, 300), (1000, 300), (255,127,0), 3)
    for (i, c) in enumerate (contours):
        (x, y, w, h) = cv2.boundingRect (c)
        validate_outline = (w >= 70) and (h >= 60)
        if not validate_outline:
            continue

        cv2.rectangle (roi, (x, y), (x + w, y + h), (0,255,0), 2)
        center = get_center (x, y, w, h)
        detections.append (center)
        cv2.circle (roi, center, 4, (0, 0,255), -1)

        for (x, y) in detections:
            if y <(300 + 6) and y> (300-6):
                vehicle += 1
                cv2.line (roi, (0, 300), (1000, 300), (0,127,255), 3)
                detections.remove ((x, y))
                print ("vehicle is detected:" + str (vehicle))


    cv2.putText (roi, "VEHICLE COUNT:" + str (vehicle), (450, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 5)
    cv2.imshow('roi', roi)


    key = cv2.waitKey(30)
    if key == 27:
        break

   

In [4]:
cap.release()
cv2.destroyAllWindows()